In [1]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 209.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 180.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 188.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 175.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 186.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0


In [2]:
!pip install git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e --upgrade
!pip install git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f --upgrade

  Cloning https://github.com/huggingface/trl (to revision a3c5b7178ac4f65569975efadc97db2f3749c65e) to /tmp/pip-req-build-qghn4u9m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-qghn4u9m
  Running command git rev-parse -q --verify 'sha^a3c5b7178ac4f65569975efadc97db2f3749c65e'
  Running command git fetch -q https://github.com/huggingface/trl a3c5b7178ac4f65569975efadc97db2f3749c65e
  Running command git checkout -q a3c5b7178ac4f65569975efadc97db2f3749c65e
  Resolved https://github.com/huggingface/trl to commit a3c5b7178ac4f65569975efadc97db2f3749c65e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.7.11.dev0-py3-none-any.whl size=151090 sha256=ae1537227715d19af99dc234d4c34c15b5ea3585b68cb5f97b125d69f7432926
  Stored in directory: /root/.cache/pip/wheels/b5/82/59/b7697a42689413e91d852fb5a1b3823c30adaab

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

In [2]:
peft_model_id = "./finetune-llama-7b-text-to-sql"
# peft_model_id = args.output_dir
 
# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

In [4]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [5]:
from datasets import load_dataset
from random import randint

In [20]:
# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))
 

In [7]:
# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [8]:
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
How many tries for does the team with 263 points against have?
Original Answer:
SELECT tries_for FROM table_name_13 WHERE points_against = "263"
Generated Answer:
SELECT tries_for FROM table_name_13 WHERE points_against = "263" AND tries_for > 0 ORDER BY tries_for DESC LIMIT 1 OFFSET 0; SELECT COUNT(tries_for) FROM table_name_13 WHERE points_against = "263" AND tries_for > 0 ORDER BY tries_for DESC LIMIT 1 OFFSET 0; SELECT COUNT(tries_for) FROM table_name_13 WHERE points_against = "263" AND tries_for > 0 ORDER BY tries_for DESC LIMIT 1 OFFSET 0; SELECT COUNT(tries_for) FROM table_name_13 WHERE points_against = "263" AND tries_for > 0 ORDER BY tries_for DESC LIMIT 1 OFFSET 0; SELECT COUNT(tries_for) FROM table_name_13 WHERE points_against = "263" AND tries_for > 0 ORDER BY tries_for DESC LIMIT 1 OFFSET 0; SELECT COUNT(tries_for) FROM table_name_13


In [29]:
eval_samples = eval_dataset.shuffle().select(range(number_of_eval_samples))

In [30]:
eval_samples

Dataset({
    features: ['messages'],
    num_rows: 1000
})

In [31]:
eval_samples[0:32].keys()

dict_keys(['messages'])

In [33]:
eval_samples[0].keys()

dict_keys(['messages'])

In [39]:
from tqdm import tqdm
 
 
def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        print('Model Result')
        print(predicted_answer)
        print('Actual Result')
        print(sample["messages"][2]["content"])
        return 0
 
success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))
    accuracy = sum(success_rate)/len(success_rate)
    print(f"Accuracy: {accuracy*100:.2f}%")
    
 
# compute accuracy
accuracy = sum(success_rate)/len(success_rate)
 
print(f"Accuracy: {accuracy*100:.2f}%")

  0%|          | 1/1000 [00:09<2:36:19,  9.39s/it]

Model Result
SELECT nationality FROM table_1473672_6 WHERE player = "Derek Black" AND nationality!= "United States" AND nationality!= "Turkey" AND nationality!= "Canada" AND nationality!= "Australia" AND nationality!= "Netherlands" AND nationality!= "Finland" AND nationality!= "Belgium" AND nationality!= "Romania" AND nationality!= "South Africa" AND nationality!= "New Zealand" AND nationality!= "Norway" AND nationality!= "Germany" AND nationality!= "Spain" AND nationality!= "United Kingdom" AND nationality!= "Hungary" AND nationality!= "Ireland" AND nationality!= "Sweden" AND nationality!= "Denmark" AND nationality!= "Luxembourg" AND nationality!= "Latvia" AND nationality!= "Lithuania" AND nationality!= "Switzerland" AND nationality!= "United Arab Emirates" AND nationality!= "Honduras" AND nationality!= "Jamaica" AND nationality!= "Croatia" AND nationality!=
Actual Result
SELECT nationality FROM table_1473672_6 WHERE player = "Derek Black"
Accuracy: 0.00%


  0%|          | 2/1000 [00:18<2:33:47,  9.25s/it]

Model Result
SELECT replaced_by FROM table_22640051_3 WHERE date_of_appointment = "8 July 2009" AND replaced_by <> "acting" AND replaced_by <> "Acting" AND replaced_by <> "Acting Minister" AND replaced_by <> "Acting Minister for Agriculture" AND replaced_by <> "Acting Minister for Health" AND replaced_by <> "Acting Minister for Immigration and Citizenship" AND replaced_by <> "Acting Minister for the Environment" AND replaced_by <> "Acting Minister for Youth Affairs and Sports" AND replaced_by <> "Acting Minister for Foreign Affairs" AND replaced_by <> "Acting Minister for Finance and Economic Development" AND replaced_by <> "Acting Minister for Justice and Home Affairs" AND replaced_by <> "Acting Minister for Primary Industries" AND replaced_by <> "Acting Minister for Social Development" AND replaced_by <> "Acting Minister for Treasury" AND replaced_by <> "Acting Minister for Women" AND replaced_by <> "Minister" AND replaced_by <> "Minister for Agriculture" AND replaced
Actual Result
S

  0%|          | 3/1000 [00:27<2:32:45,  9.19s/it]

Model Result
SELECT in_service FROM table_name_52 WHERE versions = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l-410uvp" AND "l-410uvp" = "l
Actual Result
SELECT in_service FROM table_name_52 WHERE versions = "l-410uvp"
Accuracy: 0.00%


  0%|          | 3/1000 [00:34<3:09:25, 11.40s/it]


KeyboardInterrupt: 

In [15]:
dataset_eval = eval_dataset.shuffle().select(range(number_of_eval_samples))
dataset_eval = [pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True) for sample in dataset_eval]

In [18]:
dataset_eval[80]

'<|im_start|>system\nYou are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_15 (away VARCHAR, home VARCHAR)<|im_end|>\n<|im_start|>user\nWHAT IS THE AWAY WITH HOME AT 2-2?<|im_end|>\n<|im_start|>assistant\n'

'<|im_start|>system\nYou are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_62 (rank INTEGER, player VARCHAR)<|im_end|>\n<|im_start|>user\nWhich Rank has a Player of matt ruth?<|im_end|>\n<|im_start|>assistant\n'